In [1]:
import numpy as np
from pandas import read_csv
from sklearn import preprocessing, neighbors,svm
#from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn. ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

In [2]:
train_set=read_csv('data/train_set.csv')
val_set=read_csv('data/val_set.csv')
test_set=read_csv('data/test_set.csv')

In [3]:
trainy=read_csv('data/trainy.csv')
valy=read_csv('data/valy.csv')
testy=read_csv('data/testy.csv')

In [4]:
df = read_csv('data/orginal_data.csv')

In [5]:
# make the range between 0 and 1
column = list(train_set.columns.values)
for col in column:
    minimum = df[col].min()
    maximum = df[col].max()
    train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
    val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
    test_set[col] = (test_set[col] - minimum)/(maximum-minimum)

In [6]:
train_set = np.array(train_set)
trainy= np.array(trainy).ravel()

In [7]:
#########################training decision tree#########################

In [8]:
from sklearn.model_selection import GridSearchCV
# defining parameter range
param_grid_dt = {'max_depth': [2, 5, 10, 15, 20, 25]}
 
grid = GridSearchCV(DecisionTreeClassifier(class_weight={0: 3., 1: 1.}), param_grid_dt, refit=True, verbose=1)
 
# fitting the model for grid search
grid.fit(train_set, trainy)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   25.6s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                              class_weight={0: 3.0, 1: 1.0},
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             para

In [9]:
print(grid.best_params_)

{'max_depth': 10}


In [10]:
DTC = DecisionTreeClassifier(max_depth = 10, class_weight={0: 3., 1: 1.})

In [27]:
aucs_dtc = []
for i in range(20):
    DTC.fit(train_set, trainy)
    roc_auc_dtc = roc_auc_score(testy, DTC.predict(test_set))  
    aucs_dtc.append(roc_auc_dtc)
mean_auc_dtc = np.mean(aucs_dtc)
std_auc_dtc = np.std(aucs_dtc)
print('Mean ROC DT(AUC = %0.3f $\pm$ %0.3f)' % (mean_auc_dtc, std_auc_dtc))

Mean ROC DT(AUC = 0.808 $\pm$ 0.004)


In [ ]:
#########################training Random FT #########################

In [13]:
RF = RandomForestClassifier(class_weight ={0: 3., 1: 1.})

In [29]:
aucs_rf = []
for i in range(20):
    RF.fit(train_set, trainy)
    roc_auc_rf = roc_auc_score(testy, RF.predict(test_set))
    aucs_rf.append(roc_auc_rf)  
mean_auc_rf = np.mean(aucs_rf)
std_auc_rf = np.std(aucs_rf)
print('Mean ROC RF(AUC = %0.3f $\pm$ %0.3f)' % (mean_auc_rf, std_auc_rf))

Mean ROC RF(AUC = 0.748 $\pm$ 0.020)


In [ ]:
#########################training Logistic regression #########################

In [44]:
LR = LogisticRegression(class_weight ={0: 3., 1: 1.}, solver='newton-cg')

In [45]:
aucs_lr = []
for i in range(20):
    LR.fit(train_set, trainy)
    roc_auc_lr = roc_auc_score(testy, LR.predict(test_set))
    aucs_lr.append(roc_auc_lr)  
mean_auc_lr = np.mean(aucs_lr)
std_auc_lr = np.std(aucs_lr)
print('Mean ROC LR(AUC = %0.3f $\pm$ %0.3f)' % (mean_auc_lr, std_auc_lr))

Mean ROC LR(AUC = 0.827 $\pm$ 0.000)


In [ ]:
#########################training SVM  #########################

In [14]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid_svm = {'kernel':('linear', 'rbf', 'sigmoid'), 'C':[1, 10], 'gamma':('scale', 'auto')}
 
grid = GridSearchCV(svm.SVC(class_weight={0: 3., 1: 1.}), param_grid_svm, refit=True, n_jobs=-1, verbose=1)
 
# fitting the model for grid search
grid.fit(train_set, trainy)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/Users/yanjia/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 75.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 147.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight={0: 3.0, 1: 1.0}, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': ('scale', 'auto'),
                         'kernel': ('linear', 'rbf', 'sigmoid')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [15]:
print(grid.best_params_)

{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


In [16]:
SV = svm.SVC(kernel='rbf', C=10, gamma = 'auto', class_weight={0: 3., 1: 1.})

In [17]:
aucs_sv = []
for i in range(20):
    SV.fit(train_set, trainy)
    roc_auc_sv = roc_auc_score(testy, SV.predict(test_set))
    aucs_sv.append(roc_auc_sv)  
mean_auc_sv = np.mean(aucs_sv)
std_auc_sv = np.std(aucs_sv)
print('Mean ROC SVM(AUC = %0.3f $\pm$ %0.3f)' % (mean_auc_sv, std_auc_sv))

Mean ROC SVM(AUC = 0.826 $\pm$ 0.000)


In [ ]:
#########################training Neural Network #########################

In [11]:
NN = MLPClassifier(hidden_layer_sizes=(512,),activation='relu',solver='adam',max_iter=500, n_iter_no_change=15)

In [12]:
aucs_nn = []
for i in range(20):
    NN.fit(train_set, trainy)
    roc_auc_nn = roc_auc_score(testy, NN.predict(test_set))
    aucs_nn.append(roc_auc_nn)  
mean_auc_nn = np.mean(aucs_nn)
std_auc_nn = np.std(aucs_nn)
print('Mean ROC NN(AUC = %0.3f $\pm$ %0.3f)' % (mean_auc_nn, std_auc_nn))

Mean ROC NN(AUC = 0.784 $\pm$ 0.015)


In [18]:
std_auc_nn *2, 

0.030753575530154696